# Step to test  

- Check tasks.py for basic setup
- Run the below command
```bash
# docker-compose terminal
docker-compose -f docker-compose.yml up
```

```bash
# celery terminal
celery -A celery_workers.controller.celery_app worker -l info
```

In [1]:
from celery_workers.tasks import start_pipeline

In [2]:
urls = [
    "https://docs.celeryq.dev/en/stable/getting-started/introduction.html",
    "https://docs.celeryq.dev/en/stable/getting-started/first-steps-with-celery.html",
    "https://lip17.medium.com/hands-on-learn-python-celery-in-30-minutes-9544aabb70b1",
    "https://dev.to/tecnosam/celery-pythons-silent-hero-4839",
    "https://en.wikipedia.org/wiki/Celery_(software)",
    "https://docs.celeryq.dev/en/stable/getting-started/introduction.html",
]

start_pipeline(urls).get()

'Delivered'

In [3]:
from celery_workers.agents_demo import start_agent

In [4]:
initial_context = {"history": [], "query": "How to test Celery agents?"}
start_agent.apply_async(args=[initial_context])
print("Autonomous agent chain launched")

Autonomous agent chain launched
